In [ ]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/12F9epgBqkVCww8VcBeD2S2CIfaLNe0zC?usp=share_link"
gdown.download_folder(url, quiet=True)

In [2]:
!pip install tqdm
!pip install dlib

In [1]:
import os
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import shutil
import zipfile
import dlib
from tqdm import tqdm

shape_predictor_path = "/content/biometric_systems_project/shape_predictor_5_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

def shape_to_normal(shape):
  shape_normal = []
  # shape[0] = right eye, right corner
  # shape[1] = right eye, left corner
  # shape[2] = left eye, left corner
  # shape[3] = left eye, right corner
  # shape[4] = nose
  for i in range(0, shape.num_parts):
    shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
  return shape_normal

def get_eyes_nose_dlib(shape):
  nose = shape[4][1]
  left_eye_x = (shape[3][1][0] + shape[2][1][0]) // 2
  left_eye_y = (shape[3][1][1] + shape[2][1][1]) // 2
  right_eyes_x = (shape[1][1][0] + shape[0][1][0]) // 2
  right_eyes_y = (shape[1][1][1] + shape[0][1][1]) // 2

  return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a

def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy

def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False

def get_eyes_in_image(image, top, bottom, left, right, left_eye, height_face):
  left_eye_y = left_eye[1]
  while left_eye_y > bottom:
    bottom += int(height_face * 0.02)
  image = image[top:bottom, left:right]
  return image, bottom

def get_nose_in_image(image, top, bottom, left, right, nose, height_face):
  nose_y = nose[1]
  while nose_y > bottom:
    bottom += int(height_face * 0.02)
  image = image[top:bottom, left:right]
  return image, bottom

def get_mouth_in_image(image, top, bottom, left, right):
  image = image[top:bottom, left:right]
  return image

def align_face(image, folder_path, file_name):
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = detector(gray, 0)
  if len(faces) > 0:
    for face in faces:
      # Extract face bounding box coordinates
      shape = predictor(gray, face)

      # Get facial landmarks for the detected face
      shape = shape_to_normal(shape)

      #for _, (lx, ly) in shape:
      #  cv2.circle(image, (lx, ly), radius=3, color=(255, 0, 0), thickness=-1)  # Blue dots

      # Get the nose, left eye, and right eye positions
      nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
      #cv2.circle(image, nose, radius=5, color=(0, 255, 0), thickness=-1)  # Green dot for the nose
      #cv2.circle(image, left_eye, radius=5, color=(0, 0, 255), thickness=-1)  # Red dot for left eye
      #cv2.circle(image, right_eye, radius=5, color=(0, 0, 255), thickness=-1)  # Red dot for right eye

      delta_x = right_eye[0] - left_eye[0]
      delta_y = right_eye[1] - left_eye[1]
      # angle from radians to degrees
      angle = math.atan2(delta_y, delta_x) * (180 / math.pi)

      center_of_eyes = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
      rotation_matrix = cv2.getRotationMatrix2D(center_of_eyes, angle, scale=1.0)

      h, w = image.shape[:2]

      aligned_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
      #cv2.rectangle(aligned_image, (face.left(), face.top()), (face.right(), face.bottom()), (255, 255, 255), 2)

      # catch features in the image
      height_face = face.bottom() - face.top()
      eyes_bottom = face.top() + int(height_face * 0.4)
      nose_bottom = eyes_bottom +int(height_face * 0.2)
      eyes_image, eyes_bottom = get_eyes_in_image(aligned_image, face.top(), eyes_bottom, face.left(), face.right(), left_eye, height_face)
      nose_image, nose_bottom = get_nose_in_image(aligned_image, eyes_bottom, nose_bottom, face.left(), face.right(), nose, height_face)
      mouth_image = get_mouth_in_image(aligned_image, nose_bottom, face.bottom(), face.left(), face.right())

      # save the catched features into image
      os.makedirs("eyes", exist_ok=True)
      os.makedirs("nose", exist_ok=True)
      os.makedirs("mouth", exist_ok=True)
      if eyes_image.size == 0:
        continue
      if nose_image.size == 0:
        continue
      if mouth_image.size == 0:
        continue
      cv2.imwrite(os.path.join("eyes", "eyes_" + file_name), eyes_image)
      cv2.imwrite(os.path.join("nose", "nose_" + file_name), nose_image)
      cv2.imwrite(os.path.join("mouth", "mouth_" + file_name), mouth_image)

      # boolean=True, image is aligned
      return aligned_image, True
  # boolean=False, image is not aligned
  return image, False

def main():
  folder_path = "/content/biometric_systems_project"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  aligned_faces, total_faces = 0, 0
  dataset_path = folder_path + "/dataset/"
  for file_name in tqdm(os.listdir(dataset_path), desc="Processing Images"):
    if not file_name.endswith(".jpg"):
      continue
    total_faces += 1
    if file_name.startswith("."):
      file_name = file_name[2:]
    image = cv2.imread(os.path.join(dataset_path, file_name))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    aligned_face, boolean = align_face(image, folder_path, file_name)

if __name__ == "__main__":
  main()